In [1]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever


In [2]:

local_path = "docs/Dastan.pdf"  # Убедитесь, что путь правильный

if local_path:
    loader = UnstructuredPDFLoader(file_path=local_path)
    data = loader.load()
else:
    print("Keine PDF.")


In [3]:
data[0].page_content

'Dastan Sharshekeev Kirgisisch-Deutsches Institut für Angewandte Informatik # dastan.sharshekeevv@gmail.com # dastan.sharshekeev@inai.kg § GitHub Profile (cid:239) LinkedIn Profile\n\nShul-Und Berufsbildung\n\nKirgisisch-Deutsches Institut für Angewandte Informatik\n\nAngewandte Informatik\n\nBildungskomplex. Gymnasium Schule Nr. 29\n\nBerufsehrfahrung\n\nHigh Technology Park of Kyrgyz Republic\n\nIT-Spezialist – Automatisierung des Eingangs von Einwohneranträgen. – Digitale Dokumentenverwaltung\n\nProjektarbeit\n\nFull Stack Developer – Website-Visitenkarte für den Währungsumtausch – Telegram-Chatbot für die Annahme von Bewerbungen\n\nFreiberufliche Tätigkeit\n\nFull Stack Developer – Website eines Kosmetikgeschäfts – Automatisierung des Empfangs von Briefen für die Postorganisation\n\nHausarbeit\n\nSmart Reports\n\nProjekt, das sich an kleine und mittlere Unternehmen richtet.\n\n– Verwendete Tools & Technologien: Java (Spring boot), Kotlin(Android), Heroku(DevOps) – Es ermöglicht Ihn

In [4]:
# !ollama pull nomic-embed-text

In [5]:
!ollama list

NAME                       ID              SIZE      MODIFIED       
mistral:latest             f974a74358d6    4.1 GB    39 minutes ago    
nomic-embed-text:latest    0a109f422b47    274 MB    2 hours ago       


In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)
print(chunks)

[Document(metadata={'source': 'docs/Dastan.pdf'}, page_content='Dastan Sharshekeev Kirgisisch-Deutsches Institut für Angewandte Informatik # dastan.sharshekeevv@gmail.com # dastan.sharshekeev@inai.kg § GitHub Profile (cid:239) LinkedIn Profile\n\nShul-Und Berufsbildung\n\nKirgisisch-Deutsches Institut für Angewandte Informatik\n\nAngewandte Informatik\n\nBildungskomplex. Gymnasium Schule Nr. 29\n\nBerufsehrfahrung\n\nHigh Technology Park of Kyrgyz Republic\n\nIT-Spezialist – Automatisierung des Eingangs von Einwohneranträgen. – Digitale Dokumentenverwaltung\n\nProjektarbeit\n\nFull Stack Developer – Website-Visitenkarte für den Währungsumtausch – Telegram-Chatbot für die Annahme von Bewerbungen\n\nFreiberufliche Tätigkeit\n\nFull Stack Developer – Website eines Kosmetikgeschäfts – Automatisierung des Empfangs von Briefen für die Postorganisation\n\nHausarbeit\n\nSmart Reports\n\nProjekt, das sich an kleine und mittlere Unternehmen richtet.\n\n– Verwendete Tools & Technologien: Java (Sp

In [7]:
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text", show_progress=True),
    collection_name="local-rag"
)
# print(vector_db)

/var/folders/y8/ndh9qhzj7t98rv05wws177ph0000gn/T/ipykernel_27294/2106810776.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding=OllamaEmbeddings(model="nomic-embed-text", show_progress=True),
OllamaEmbeddings: 100%|██████████| 1/1 [00:01<00:00,  1.23s/it]


In [8]:

# !ollama pull mistral

In [9]:
local_model = "mistral"
llm = ChatOllama(model=local_model)

/var/folders/y8/ndh9qhzj7t98rv05wws177ph0000gn/T/ipykernel_27294/1632367389.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model=local_model)


In [10]:
QUERY_PROMPT = PromptTemplate(
    input_variables = ['question'],
    template=""" You are an intelligent model designed to help the user answer questions. Your task is to provide answers to questions based only on the information contained in the uploaded PDF document. You must answer in German. If there is no information in the document, report that there is no answer. Original question: {question}. After answering to question. Forget all information and data from PDF.
   """,

)

In [11]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(),
    llm,
    prompt=QUERY_PROMPT
)
# print(retriever)
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)


In [12]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [16]:
chain.invoke(input(""))

{
  context: MultiQueryRetriever(retriever=VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x109e137c0>, search_kwargs={}), llm_chain=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template=' You are an intelligent model designed to help the user answer questions. Your task is to provide answers to questions based only on the information contained in the uploaded PDF document. You must answer in German. If there is no information in the document, report that there is no answer. Original question: {question}. After answering to question. Forget all information and data from PDF.\n   ')
           | ChatOllama(model='mistral')
           | LineListOutputParser()),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['

In [14]:
# chain.invoke(input(""))

In [15]:
# chain.invoke(input(""))